# Objetivo

Gerar um arquivo com informação do nome da região e coordenadas geográficas para usar na visualização das recomendações de novos clientes.

# Método

A base de dados do IBGE tem as coordenadas geográficas.

Para extrair os dados (municípios e regiões) da base do IBGE: 
https://github.com/poliquin/br-localidades

(This repo downloads geo data for Brazilian localities from IBGE and converts it to CSV. The resulting file has the latitude, longitude, and altitude of places in Brazil along with the numeric identifiers from IBGE.)

Aqui é feita a leitura da base e adaptação para deixar apenas as informações necessárias.

**Importante:** como os nomes das regiões na base estaticos_market não tem acento, aqui é feita uma "limpeza", ou seja, retirada dos acentos, para compatibilizar a busca.

In [1]:
import pandas as pd

# Leitura do arquivo com latitude e longitude

O arquivo `latlng_local.csv` é obtido a partir da execução do script obtido de https://github.com/poliquin/br-localidades

In [2]:
coord_ibge = pd.read_csv('latlng_local.csv')
coord_ibge.shape

(21886, 21)

# Retirada dos acentos dos nomes das regiões

In [3]:
# agora, tem que criar um dataframe com o nome das regiões e coordenadas geográficas.
# será usada a média, ou seja, mais ou menos o centro da região,
# obtida a partir dos dados dos municípios

# o nome do estado também ficará junto.
coord_micro = coord_ibge.groupby(['nm_micro', 'nm_uf'])['lng', 'lat'].mean()

In [4]:
nomes = coord_micro.index.get_level_values(0)

In [5]:
# tirar os acentos
import unicodedata
tempdict={}
for text in nomes:
    #print(unicodedata.normalize('NFKD', text).encode('ascii', 'ignore'))
    tempdict[text]=unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

In [6]:
coord_micro.rename(tempdict, inplace=True) 
# TODO: a mudança também foi aplicada para nm_uf (SÃO PAULO ficou sem til), mas não para todos os estados.
# Verificar e corrigir caso seja feita busca pelo nome do estado.

In [7]:
coord_micro.reset_index(level=['nm_micro'], inplace=True)

In [8]:
coord_micro.head(3)

,nm_micro,lng,lat
nm_uf,,,
SAO PAULO,ADAMANTINA,-50.997688,-21.706115
ESPÍRITO SANTO,AFONSO CLAUDIO,-41.018390,-20.191878
MARANHÃO,AGLOMERACAO URBANA DE SAO LUIS,-44.142982,-2.562962


# Gravar dataframe resultante em um arquivo csv

Arquivo tem as informações:
- nm_uf (nome do estado)
- nm_micro (nome da microrregião)
- lng (longitude do "centro", obtido pela média, da microrregião)
- lat (latitude do "centro" da microrregião)

In [9]:
coord_micro.to_csv('coord_geograficas_microrregioes_brasil.csv')

---